In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)
import time


In [2]:
# 导入mnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.85s/it]


----------------------------
Train_z, Test_z
PCA: 0.946
SVD: 0.9410000000000001
NetD, Acc at first 0.9498


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.9352
SVD: 0.9303
NetG, Acc at first 0.9602


In [6]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [7]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 5

for epoch in range(num_steps):
    if 1==1:
        
        #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        #radius_list.append(radius)
        #print(radius)

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            #radius_list.append(radius)
            #print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        

tensor(-12.0842, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.4033, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.0590, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.4103, device='cuda:0', grad_fn=<AddBackward0>)]
99.97201769495192
tensor(-10.7059, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.5219, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.6213, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.2586, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-11.0764, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.5219, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.9714, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.9843, device='cuda:0', grad_fn=<AddBackward0>)]
1.3740065462559539
tensor(-18.2387, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-8.0989, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.6106, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.0992, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-19.1173, device='cu

0.44335111173066066
tensor(-50.6522, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.7282, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.0274, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.6634, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-50.7444, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.7282, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.1073, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.7452, device='cuda:0', grad_fn=<AddBackward0>)]
0.49653919190170814
tensor(-50.8966, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8273, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.0209, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.7561, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-50.9119, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8273, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.0907, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.3445, device='cuda:0', grad_fn=<AddBackward0>)]
0.4254872721436422
t

0.4184114891911153
tensor(-63.1732, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.0808, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.3727, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5827, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-63.1507, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.0808, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.3277, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.7101, device='cuda:0', grad_fn=<AddBackward0>)]
0.3904507010160912
tensor(-63.2441, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.1949, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.4193, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.1033, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-63.3239, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.1949, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.5568, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.8172, device='cuda:0', grad_fn=<AddBackward0>)]
0.41445724324637623
te

0.3968409115044761
tensor(-67.0357, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.1732, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.2516, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.0072, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-67.2569, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.1732, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.4457, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.1445, device='cuda:0', grad_fn=<AddBackward0>)]
0.39686325931347965
tensor(-68.7895, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.0598, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.0704, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.2557, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.8535, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.0598, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.2234, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.8081, device='cuda:0', grad_fn=<AddBackward0>)]
0.37559343906474274
t

0.36300828872446084
tensor(-74.4254, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-36.8653, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.1528, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.0853, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-74.2928, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-36.8653, device='cuda:0', grad_fn=<NegBackward0>), tensor(-35.9714, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.2893, device='cuda:0', grad_fn=<AddBackward0>)]
0.3523520154942909
tensor(-75.3068, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2660, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.5885, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.2729, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-75.3286, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2660, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.6604, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.0644, device='cuda:0', grad_fn=<AddBackward0>)]
0.3420518519773196
te

...

0.2895940795239449
tensor(-96.7262, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.2492, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6996, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.1757, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.7828, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.2492, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.7666, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.1531, device='cuda:0', grad_fn=<AddBackward0>)]
0.2825868796570816
tensor(-96.8054, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.2728, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.8001, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.0802, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.8068, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.2728, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.7706, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.1456, device='cuda:0', grad_fn=<AddBackward0>)]
0.29345102917174126
te

0.2987816643441508
tensor(-96.9383, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.3706, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.7690, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.2225, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.9646, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.3706, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.8159, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.1773, device='cuda:0', grad_fn=<AddBackward0>)]
0.2646711770909996
tensor(-97.0004, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.3912, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.9652, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.9040, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-97.0129, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.3912, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.9471, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.9632, device='cuda:0', grad_fn=<AddBackward0>)]
0.3072537362048742
ten

0.2986692984556286
tensor(-97.2096, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.5106, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.9324, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.1524, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-97.2848, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.5106, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.9651, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.2460, device='cuda:0', grad_fn=<AddBackward0>)]
0.29196158159859764
tensor(-97.2020, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.4916, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.9834, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.0689, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-97.2504, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.4916, device='cuda:0', grad_fn=<NegBackward0>), tensor(-48.0045, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.1263, device='cuda:0', grad_fn=<AddBackward0>)]
0.2941874728683067
te

0.2868301549962689
tensor(-97.3969, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.5886, device='cuda:0', grad_fn=<NegBackward0>), tensor(-48.1002, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.0302, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-97.4289, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.5886, device='cuda:0', grad_fn=<NegBackward0>), tensor(-48.1319, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.0309, device='cuda:0', grad_fn=<AddBackward0>)]
0.30478965584580636
tensor(-97.4493, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.5868, device='cuda:0', grad_fn=<NegBackward0>), tensor(-48.0646, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.2208, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-97.3882, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.5868, device='cuda:0', grad_fn=<NegBackward0>), tensor(-48.0276, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.1681, device='cuda:0', grad_fn=<AddBackward0>)]
0.29464042625100195
t

0.29357262832877784
tensor(-97.5819, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.6778, device='cuda:0', grad_fn=<NegBackward0>), tensor(-48.1830, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.0568, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-97.6115, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.6778, device='cuda:0', grad_fn=<NegBackward0>), tensor(-48.2003, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.0822, device='cuda:0', grad_fn=<AddBackward0>)]
0.28158664761979946
tensor(-97.5305, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.6397, device='cuda:0', grad_fn=<NegBackward0>), tensor(-48.2002, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.9949, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-97.4470, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-48.6397, device='cuda:0', grad_fn=<NegBackward0>), tensor(-48.1171, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.9940, device='cuda:0', grad_fn=<AddBackward0>)]
0.29373528529135173


In [8]:
radius_list

[99.97201769495192,
 1.3740065462559539,
 2.720376565418732,
 99.99332673960592,
 0.9401553023031095,
 0.8197327007772182,
 99.97606172681377,
 0.6594673841332338,
 0.6509235677904738,
 0.5763317012032604,
 0.5794537655440101,
 0.5806084656534953,
 0.5059994408123377,
 0.5481175875121626,
 0.5387857989600923,
 0.491182979586555,
 0.5171213458782722,
 0.44335111173066066,
 0.49653919190170814,
 0.4254872721436422,
 0.5057952783752385,
 0.4578352103303126,
 0.4573311194012386,
 0.45325994427077765,
 0.47736286480387896,
 0.4596059331508983,
 0.4304298955433022,
 0.46042786648678913,
 0.42732729316860707,
 0.43845500120198616,
 0.43771634336704873,
 0.4217775457664469,
 0.4138051177173747,
 0.44020331211621916,
 0.4184114891911153,
 0.3904507010160912,
 0.41445724324637623,
 0.4146846632411579,
 0.42580541402657696,
 0.41577699456667006,
 0.406876422877424,
 0.3985412922459072,
 0.3793193637248858,
 0.3840729493698216,
 0.39819569178514835,
 0.37185677850920545,
 0.4122694569789917,
 0.37

In [9]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.78s/it]


----------------------------
Train_z, Test_z
PCA: 0.9594
SVD: 0.9558
NetD, Acc at first 0.967


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.9594
SVD: 0.9571
NetG, Acc at first 0.9718


In [10]:
torch.save(netD, 'MNIST_NetD-mode11-radius1.pth')
torch.save(netV, 'MNIST_NetV-mode11-radius1.pth')
torch.save(netG, 'MNIST_NetG-mode11-radius1.pth')

In [11]:
#robust testing
r_list = [0,0.1,0.2,0.5,1,2,3,4,6,8,10]
sample_num = 20

In [12]:
# sample 20 times of NetG at final radius, NetD is fixed, length 0.1,0.2,0.5,1,2,4,6,8,10,15,20
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample = []
for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[1])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample.append(count/10000)

print(max(Acc_results_G_sample),min(Acc_results_G_sample),np.mean(Acc_results_G_sample),np.var(Acc_results_G_sample))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.9584
SVD: 0.9544
NetG_fixD, Acc at sample 1 0.9698


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9579
SVD: 0.9544
NetG_fixD, Acc at sample 2 0.9699


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9578
SVD: 0.9544
NetG_fixD, Acc at sample 18 0.9697


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.958
SVD: 0.9541
NetG_fixD, Acc at sample 19 0.9701


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9578
SVD: 0.9546
NetG_fixD, Acc at sample 20 0.9702
0.9703 0.9696 0.9699899999999998 3.089999999999663e-08


In [13]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample2 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[2])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample2.append(count/10000)
    
print(max(Acc_results_G_sample2),min(Acc_results_G_sample2),np.mean(Acc_results_G_sample2),np.var(Acc_results_G_sample2))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9574
SVD: 0.9537
NetG_fixD, Acc at sample 1 0.9699


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.9584
SVD: 0.9543
NetG_fixD, Acc at sample 2 0.9702


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9578
SVD: 0.954
NetG_fixD, Acc at sample 18 0.9699


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9576
SVD: 0.9545
NetG_fixD, Acc at sample 19 0.97


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.9586
SVD: 0.9547
NetG_fixD, Acc at sample 20 0.9701
0.9704 0.9695 0.9700099999999997 3.889999999999898e-08


In [14]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample3 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[3])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample3.append(count/10000)
    
print(max(Acc_results_G_sample3),min(Acc_results_G_sample3),np.mean(Acc_results_G_sample3),np.var(Acc_results_G_sample3))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9578
SVD: 0.9549
NetG_fixD, Acc at sample 1 0.9698


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9575
SVD: 0.9542
NetG_fixD, Acc at sample 2 0.9697


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9568
SVD: 0.9531000000000001
NetG_fixD, Acc at sample 18 0.9701


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9589
SVD: 0.954
NetG_fixD, Acc at sample 19 0.969


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.78s/it]


----------------------------
Train_z, Test_z
PCA: 0.9562
SVD: 0.9538
NetG_fixD, Acc at sample 20 0.9694
0.9705 0.9687 0.96951 3.1690000000000724e-07


In [15]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample4 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[4])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample4.append(count/10000)
    
print(max(Acc_results_G_sample4),min(Acc_results_G_sample4),np.mean(Acc_results_G_sample4),np.var(Acc_results_G_sample4))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9558
SVD: 0.9534
NetG_fixD, Acc at sample 1 0.968


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9553
SVD: 0.9519
NetG_fixD, Acc at sample 2 0.9687


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9563
SVD: 0.9516
NetG_fixD, Acc at sample 18 0.9671


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9569
SVD: 0.9545
NetG_fixD, Acc at sample 19 0.9678


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9557
SVD: 0.953
NetG_fixD, Acc at sample 20 0.9684
0.969 0.9657 0.9678849999999999 4.3427500000000245e-07


In [16]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample5 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[5])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample5.append(count/10000)
    
print(max(Acc_results_G_sample5),min(Acc_results_G_sample5),np.mean(Acc_results_G_sample5),np.var(Acc_results_G_sample5))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9501
SVD: 0.9483
NetG_fixD, Acc at sample 1 0.957


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.9522
SVD: 0.9474
NetG_fixD, Acc at sample 2 0.9583


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.948
SVD: 0.9464
NetG_fixD, Acc at sample 18 0.9593


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9524
SVD: 0.9508
NetG_fixD, Acc at sample 19 0.9582


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9469
SVD: 0.9452
NetG_fixD, Acc at sample 20 0.9605
0.9617 0.9567 0.959435 1.5142750000000072e-06


In [17]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample6 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[6])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample6.append(count/10000)
    
print(max(Acc_results_G_sample6),min(Acc_results_G_sample6),np.mean(Acc_results_G_sample6),np.var(Acc_results_G_sample6))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.9404
SVD: 0.9359999999999999
NetG_fixD, Acc at sample 1 0.9388


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.75s/it]


----------------------------
Train_z, Test_z
PCA: 0.9372
SVD: 0.9323
NetG_fixD, Acc at sample 2 0.849


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9406
SVD: 0.9341
NetG_fixD, Acc at sample 18 0.947


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.9426
SVD: 0.9381
NetG_fixD, Acc at sample 19 0.9475


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9409
SVD: 0.94
NetG_fixD, Acc at sample 20 0.9356
0.9482 0.849 0.9250150000000001 0.0005570972750000001


In [18]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample7 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[7])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample7.append(count/10000)
    
print(max(Acc_results_G_sample7),min(Acc_results_G_sample7),np.mean(Acc_results_G_sample7),np.var(Acc_results_G_sample7))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9381
SVD: 0.9354
NetG_fixD, Acc at sample 1 0.5939


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.9297
SVD: 0.9255
NetG_fixD, Acc at sample 2 0.4958


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9308
SVD: 0.9285
NetG_fixD, Acc at sample 18 0.774


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.9243
SVD: 0.9214
NetG_fixD, Acc at sample 19 0.5666


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.9238999999999999
SVD: 0.9207
NetG_fixD, Acc at sample 20 0.6052
0.774 0.3717 0.5643349999999999 0.010529928275000001


In [19]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample8 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[8])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample8.append(count/10000)
    
print(max(Acc_results_G_sample8),min(Acc_results_G_sample8),np.mean(Acc_results_G_sample8),np.var(Acc_results_G_sample8))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.9293
SVD: 0.9254
NetG_fixD, Acc at sample 1 0.1745


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


----------------------------
Train_z, Test_z
PCA: 0.9235
SVD: 0.9186
NetG_fixD, Acc at sample 2 0.2124


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9238
SVD: 0.92
NetG_fixD, Acc at sample 18 0.1548


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9211
SVD: 0.9172
NetG_fixD, Acc at sample 19 0.1518


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9262
SVD: 0.9227
NetG_fixD, Acc at sample 20 0.2031
0.2166 0.104 0.16093000000000002 0.0009455530999999999


In [20]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample9 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[9])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample9.append(count/10000)
    
print(max(Acc_results_G_sample9),min(Acc_results_G_sample9),np.mean(Acc_results_G_sample9),np.var(Acc_results_G_sample9))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9229
SVD: 0.9189
NetG_fixD, Acc at sample 1 0.121


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9265
SVD: 0.9236
NetG_fixD, Acc at sample 2 0.1049


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9278
SVD: 0.9266
NetG_fixD, Acc at sample 18 0.1273


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


----------------------------
Train_z, Test_z
PCA: 0.9298
SVD: 0.926
NetG_fixD, Acc at sample 19 0.071


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9286
SVD: 0.9253
NetG_fixD, Acc at sample 20 0.1161
0.1426 0.071 0.11297500000000002 0.00030649087500000006


In [21]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample10 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[10])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample10.append(count/10000)
    
print(max(Acc_results_G_sample10),min(Acc_results_G_sample10),np.mean(Acc_results_G_sample10),np.var(Acc_results_G_sample10))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9281
SVD: 0.9251
NetG_fixD, Acc at sample 1 0.1029


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.9369
SVD: 0.9332
NetG_fixD, Acc at sample 2 0.0957


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.85s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9228
SVD: 0.9178
NetG_fixD, Acc at sample 18 0.0835


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9362
SVD: 0.9318
NetG_fixD, Acc at sample 19 0.0955


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.77s/it]


----------------------------
Train_z, Test_z
PCA: 0.931
SVD: 0.9287
NetG_fixD, Acc at sample 20 0.1036
0.1308 0.0745 0.10457999999999998 0.0002198855999999999
